# Web scraping AirBnB

In [21]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Get current working directory
print(os.getcwd())

/Users/andreas/Nextcloud/cloud_atrip/Dokumente/Repos/applied-data-science


# Url to scrape

In [22]:
# urlStringHome = 'https://www.airbnb.ch/'
# urlString = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt"
urlStringFlat0to68 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=9&price_max=68"
urlStringFlat69to84 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=69&price_max=84"
urlStringFlat85to100 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=85&price_max=100"
urlStringFlat101to114 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=101&price_max=114"
urlStringFlat115to132 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=115&price_max=132"
urlStringFlat133to150 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=133&price_max=150"
urlStringFlat151to173 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=151&price_max=173"
urlStringFlat174to202 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=151&price_max=173"
urlStringFlat203to245 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=203&price_max=245"
urlStringFlat246to316 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=246&price_max=316"
urlStringFlat317to500 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=317&price_max=500"
urlStringFlat501to1000 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=501&price_max=1000"

In [23]:
# Set options for Chrome-Driver (headless mode)
opts = Options()
opts.add_argument("--window-size=1200,800")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# Website to scrape
driver.get(urlStringFlat69to84)

# Get current url
print(driver.current_url)

# Return the full page HTML code
# print(driver.page_source)

# Gets the page's title
print(driver.title)

# Close driver
# driver.close()

https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=69&price_max=84
Erstklassige Unterkünfte & Ferienunterkünfte in Zürich: Gesamte Unterkunft | Airbnb - Airbnb


In [24]:
#time.sleep(5)
# wait till cookie banner pops up
try:
    # close cookie banner
    cookie = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html[@class='js-focus-visible']/body[@class='with-new-header']/div[5]/div/div/div/div[1]/div[3]/section/div[2]/div[2]/button"))) 
    cookie.click()
except:
    pass

In [25]:
# get entries of apparements on page
nr_appartements = len(driver.find_elements(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div"))
nr_appartements

18

## Fields to scrape from detail appartement page

* Kommentare
* Preis
* Standort 
* Gesamt Rating
  * Alle Subfelder Sauberkeit, Kommunikation etc.
* Anz. Gäste
* Anz. Schlafzimmer 
* Anz. Better
* Anz. Badezimmer 
* Anz. Bewertungen
* Wo du schlafen wirst

In [38]:
# scrapping logic for detail page apartment
def scrap_appartement(nr_of_flat):

    XPATH_cleanliness = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[1]/div/div/div[2]/span"
    XPATH_communication = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[3]/div/div/div[2]/span/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[3]/div/div/div[2]/span"
    XPATH_checkin = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[5]/div/div/div[2]/span"
    XPATH_valueformoney = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[6]/div/div/div[2]/span"
    XPATH_location = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[2]/div/div/div[2]/span"
    XPATH_data_accuracy = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[2]/div/div/div[2]/span"    
    rating_cleanliness = ""
    rating_communication = ""
    rating_checkin = ""
    rating_valueformoney = ""
    rating_location = ""
    rating_data_accuracy = ""
    
    XPATH_APARETMENT_NAME = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[1]/span/h1"
    apartment_name = ""

    XPATH_PRICE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[1]"
    apartment_price = ""
    XPATH_PRICE_REDUCED = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[2]"
    apartment_price_reduced = ""

    XPATH_PLACE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[3]/button/span"
    XPATH_PLACE_2 = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[5]/button/span"
    apartment_place = ""

    XPATH_RATING = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[1]/span[2]"
    rating = ""

    print(nr_of_flat)
    # check if page title is present
    try:
        element_present = EC.presence_of_element_located((By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[1]/span/h1"))
        WebDriverWait(driver, 6).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        return
    
    try:
        apartment_name = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.XPATH, XPATH_APARETMENT_NAME))).text
    except:
        pass

    try:
        apartment_price = driver.find_element(By.XPATH, XPATH_PRICE).text

        if driver.find_elements(By.XPATH, XPATH_PRICE_REDUCED):
            apartment_price_reduced = driver.find_element(By.XPATH, XPATH_PRICE_REDUCED).text
            if "CHF" not in apartment_price_reduced:
                apartment_price_reduced = ""
    except:
        pass

    try:
        if driver.find_elements(By.XPATH, XPATH_PLACE):
            apartment_place = driver.find_element(By.XPATH, XPATH_PLACE).text
        else:
            apartment_place = driver.find_element(By.XPATH, XPATH_PLACE_2).text
    except:
        pass

    try:
        rating = driver.find_element(By.XPATH, XPATH_RATING).text
        rating_checkin = driver.find_element(By.XPATH, XPATH_checkin).text
        rating_cleanliness = driver.find_element(By.XPATH, XPATH_cleanliness).text
        rating_communication = driver.find_element(By.XPATH, XPATH_communication).text
        rating_data_accuracy = driver.find_element(By.XPATH, XPATH_data_accuracy).text
        rating_location = driver.find_element(By.XPATH, XPATH_location).text
        rating_valueformoney = driver.find_element(By.XPATH, XPATH_valueformoney).text
    except:
        pass
        
    return {'name': [apartment_name], 'price': [apartment_price],'price_reduced': [apartment_price_reduced], 'place': [apartment_place], 'rating_overall': [rating], 'rating_valueformoney': [rating_valueformoney], 'rating_location': [rating_location], 'rating_checkin': [rating_checkin], 'rating_communication': [rating_communication], 'rating_cleanliness': [rating_cleanliness], 'rating_data_accuracy': [rating_data_accuracy]}

In [39]:
def handle_appartements_tabs(nr_appartements):
    # Click on each appartement which is opened in seperate tab
    apartments = pd.DataFrame()
    for nr in range(1, nr_appartements):
        xpath = f"/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div[{nr}]"
        driver.find_element(By.XPATH, xpath).click()

        try:
            # switch tab
            #get old tab
            parent = driver.window_handles[0]
            #get new tab
            chld = driver.window_handles[1]
            #switch to new tab
            driver.switch_to.window(chld)
        except:
            print("could not swtich tab")

        if nr == 1:
            try:
                # close translate banner
                transl = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[9]/section/div/div/div[2]/div/div[1]/button"))) 
                transl.click()
            except:
                pass
        
        apartment_scraped = scrap_appartement(nr)
        apartments = pd.concat([apartments, pd.DataFrame(apartment_scraped)])
        
        try: 
            # close tab
            parent = driver.window_handles[0]
            chld = driver.window_handles[1]
            driver.switch_to.window(chld)
            driver.close()
            time.sleep(0.5)
            parent = driver.window_handles[0]
            driver.switch_to.window(parent)
        except:
            print("could not close tab")
    
    return apartments
    

In [37]:
#for testing...
apa_test = handle_appartements_tabs(8)
apa_test

1
2
3
4
5
6
7


,name,price,price_reduced,place,rating_overall,rating_valueformoney,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy
0,Stadt Zürich15/ kleines Studio,62 CHF,,"Zürich, Schweiz","4,86 ·",,,"5,0",,"4,9",
0,Goldiges See Apartment,,,"Küsnacht, Zürich, Schweiz",Neu,,,,,,
0,"30min Zürich, Baden, Aarau, Luzern",54 CHF,,"Kallern, Aargau, Schweiz","5,0 ·",,,,,,
0,Cosy studio close to hot spots,82 CHF,74 CHF,"Zürich, Schweiz","4,60 ·",,,"4,7",,"4,8",
0,Affoltern a.A. Dachwohnung/ Attic apartement,60 CHF,,"Affoltern am Albis, Zürich, Schweiz","4,54 ·",,,,,,
0,"Schickes 1-Zimmer Apartment, nur 2 Min. zum Rhein",68 CHF,,"Küssaberg, Baden-Württemberg, Deutschland","4,83 ·",,,"4,8",,"4,8",
0,❤️iges Studio im Landhausstil (40 qm2) & Sitzp...,71 CHF,,"Uster, Zürich, Schweiz","4,85 ·",,,"5,0",,"5,0",


In [ ]:
nr_pages = driver.find_element(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[3]/div/div/nav/div/a[4]").text
nr_pages = int(nr_pages)
nr_pages

In [ ]:
for nr_p in range(1, nr_pages):
    handle_appartements_tabs(nr_appartements + 1)
    if nr_p == 1:
        driver.find_element(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[3]/div/div/nav/div/a[5]").click()
    else:
        driver.find_element(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[3]/div/div/nav/div/a[6]").click()

In [ ]:
parent = driver.window_handles[0]
driver.switch_to.window(parent)
driver.close()